Complete the exercises below For **Assignment #13**.

Load the `ISLR2` and the `tidymodels` packages.

In [4]:
library('ISLR2')
library('tidymodels')

── Attaching packages ────────────────────────────────────── tidymodels 1.2.0 ──

✔ broom        1.0.5      ✔ recipes      1.0.10
✔ dials        1.2.1      ✔ rsample      1.2.1 
✔ dplyr        1.1.4      ✔ tibble       3.2.1 
✔ ggplot2      3.5.1      ✔ tidyr        1.3.1 
✔ infer        1.0.7      ✔ tune         1.2.1 
✔ modeldata    1.3.0      ✔ workflows    1.1.4 
✔ parsnip      1.2.1      ✔ workflowsets 1.1.0 
✔ purrr        1.0.2      ✔ yardstick    1.3.1 

── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ purrr::discard() masks scales::discard()
✖ dplyr::filter()  masks stats::filter()
✖ dplyr::lag()     masks stats::lag()
✖ recipes::step()  masks stats::step()
• Use tidymodels_prefer() to resolve common conflicts.



In this assignment we will use the `Default` dataset which includes the default status for credit card customers (`default` variable) in addition to each customer's:

1. credit card balance (`balance` variable),
1. student status (`student` variable), and,
1. income (`income` variable).

In [9]:
Default |> head()

,default,student,balance,income
,<fct>,<fct>,<dbl>,<dbl>
1,No,No,729.5265,44361.625
2,No,Yes,817.1804,12106.135
3,No,No,1073.5492,31767.139
4,No,No,529.2506,35704.494
5,No,No,785.6559,38463.496
6,No,Yes,919.5885,7491.559


We will be modeling `default` with the customer features.

Before we begin let's count how many customers fall into each `default` category.

In [10]:
Default |> count(default)

default,n
<fct>,<int>
No,9667
Yes,333


The data is quite imbalanced. This will be important to keep in mind when we evaluate the performance of our model later. 

Run the code below to create and training data from `Default`. We will use the "test" dataset at the end to get a final evaluation of our best model's accuracy.

In [11]:
Default_split = initial_split(Default, prop = 0.90, strata = default)

Default_train = training(Default_split)
Default_test = testing(Default_split)

Create a logistic regression model called `mod`. Set the engine to `glm` and the mode to `classification`. 

In [12]:
mod = logistic_reg(
    mode = 'classification',
    engine = 'glm')

Our data is imbalanced. As such, a naive model that *always* predicts a customer to **not default** would be correct quite often. Let's start by calculating the "accuracy" of a naive model. This will be the baseline accuracy by which we evaluate other models.

In [13]:
# This code calculates the accuracy of a model that always predicts default to be "No"

Default_train |>
    mutate(.pred_naive = factor('No', levels = c('No', 'Yes'))) |>
    accuracy(truth = default, .pred_naive)

.metric,.estimator,.estimate
<chr>,<chr>,<dbl>
accuracy,binary,0.9675556


Let's use k-fold cross validation to evaluate the performance of a model where the outcome is `default` and the predictors are `income` and `balance`.

To start, use `vfold_cv` to generate 10 validation folds (i.e. set the `v` variable to 10). Set the `strata` argument to `default` so we preserve the distribution of `default` values in each fold.

Creat your folds below and use `glimpse` to look at the output table. Call your output folds tables "folds".

In [14]:
folds = vfold_cv(Default, v = 10, strata = default)

folds |> glimpse()

Rows: 10
Columns: 2
$ splits <list> [<vfold_split[9000 x 1000 x 10000 x 4]>], [<vfold_split[9000 x…
$ id     <chr> "Fold01", "Fold02", "Fold03", "Fold04", "Fold05", "Fold06", "Fo…


The code below fits a model to each of your 10 folds. `collect_metrics` finds the average of evaluation metrics for each of your ten models. 

In [15]:
mod |> 
    fit_resamples(default ~ income + balance, folds) |>
    collect_metrics()

.metric,.estimator,mean,n,std_err,.config
<chr>,<chr>,<dbl>,<int>,<dbl>,<chr>
accuracy,binary,0.97360000,10,0.001117537,Preprocessor1_Model1
brier_class,binary,0.02145916,10,0.001086090,Preprocessor1_Model1
roc_auc,binary,0.94876222,10,0.005821776,Preprocessor1_Model1


❓How does the model accuracy compare to the naive model from above?

The naive model from above had and accuracy of ~.9676 and the average accuracy for a 10 folds is .9736.  Not significantly greater, but including balance and income as predictors increases our model accuracy. 

Complete the cell below to evaluate a model also includes the `student` variable as as predictor.
1. use `default ~ income + balance + student` as the formula,
2. encode your `student` variable with `step_dummy`, and,
3. don't forget to `prep` your recipe!

In [17]:
rec = recipe(default ~ income + balance + student, data = Default) |>
    step_dummy(student) |>
    prep()

mod |> 
    fit_resamples(default ~ income + balance + student, folds) |>
    collect_metrics()

.metric,.estimator,mean,n,std_err,.config
<chr>,<chr>,<dbl>,<int>,<dbl>,<chr>
accuracy,binary,0.97310000,10,0.001361780,Preprocessor1_Model1
brier_class,binary,0.02138019,10,0.001077471,Preprocessor1_Model1
roc_auc,binary,0.94883616,10,0.005915629,Preprocessor1_Model1


❓Does it appear that the model that includes `student` improves upon the first model with only `income` and `balance` as predictors?

Comparing to the model including only income and balance as predictors, the model with student added has a slightly lower accuracy (.9736 > .9731).  This would indicate that the earlier model (w/o student) is more accurate and a better fit.

Finally, estimate the accuracy of an `default ~ income + balance` model on the test data, `Default_test`. 

❓Does our model outperform a naive model?

In [116]:
mod_fit = mod |> fit(default ~ income + balance, data = Default) 

tidy(mod_fit)

pred = augment(mod_fit, Default_test)

pred |> glimpse()

term,estimate,std.error,statistic,p.value
<chr>,<dbl>,<dbl>,<dbl>,<dbl>
(Intercept),-1.154047e+01,4.347564e-01,-26.544680,2.958355e-155
income,2.080898e-05,4.985167e-06,4.174178,2.990638e-05
balance,5.647103e-03,2.273731e-04,24.836280,3.638120e-136


Rows: 1,000
Columns: 7
$ .pred_class <fct> No, No, No, No, No, No, No, No, No, No, No, No, No, No, No…
$ .pred_No    <dbl> 0.9987381, 0.9919738, 0.9955272, 0.9425073, 0.9708034, 0.9…
$ .pred_Yes   <dbl> 1.261930e-03, 8.026211e-03, 4.472807e-03, 5.749274e-02, 2.…
$ default     <fct> No, No, No, No, No, No, No, No, No, No, No, No, No, No, No…
$ student     <fct> Yes, No, No, Yes, No, Yes, No, No, No, No, No, Yes, No, No…
$ balance     <dbl> 817.18041, 1073.54916, 913.58717, 1499.72466, 1238.61032, …
$ income      <dbl> 12106.13, 31767.14, 46907.23, 13190.65, 50066.68, 13120.64…


In [115]:
accuracy = (count(pred, .pred_class == default)/1000) |>
print()

count(pred, .pred_class == 'No')

  .pred_class == default     n
1                  0.000 0.034
2                  0.001 0.966


".pred_class == ""No""",n
<lgl>,<int>
FALSE,23
TRUE,977


Calculating accuracy of our model fitted against Default_test predictions we can see that our accuracy is .966 which is very close to the accuracy of the naive model. 

It does not outperform our naive model.  Building our model on the Default data (not on the test split) we can see below that our accuracy increases to .9737 which is greater than our naive model.  I would still suggest using income and balance as predictors for a better fitting model.

In [119]:
mod_fit = mod |> fit(default ~ income + balance, data = Default) 

pred = augment(mod_fit, Default)

pred |> glimpse()

accuracy = (count(pred, .pred_class == default)/10000) |>
print()

count(pred, .pred_class == 'No')

Rows: 10,000
Columns: 7
$ .pred_class <fct> No, No, No, No, No, No, No, No, No, No, No, No, No, No, No…
$ .pred_No    <dbl> 0.9984953, 0.9987381, 0.9919738, 0.9995940, 0.9981733, 0.9…
$ .pred_Yes   <dbl> 1.504728e-03, 1.261930e-03, 8.026211e-03, 4.059957e-04, 1.…
$ default     <fct> No, No, No, No, No, No, No, No, No, No, No, No, No, No, No…
$ student     <fct> No, Yes, No, No, No, Yes, No, Yes, No, No, Yes, Yes, No, N…
$ balance     <dbl> 729.5265, 817.1804, 1073.5492, 529.2506, 785.6559, 919.588…
$ income      <dbl> 44361.625, 12106.135, 31767.139, 35704.494, 38463.496, 749…
  .pred_class == default      n
1                  0e+00 0.0263
2                  1e-04 0.9737


".pred_class == ""No""",n
<lgl>,<int>
FALSE,146
TRUE,9854
